In [8]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [39]:
def rand_init(shape):
    return tf.Variable(tf.random_normal(shape,mean=0.0,stddev=0.01))
tf.reset_default_graph()
x_ = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])

W1 = rand_init([784,64])
b1 = rand_init([64])
layer1 = tf.nn.dropout(tf.tanh(tf.matmul(x_,W1)+b1),keep_prob=0.8)

W2 = rand_init([64,64])
b2 = rand_init([64])
layer2 = tf.nn.dropout(tf.tanh(tf.matmul(layer1,W2)+b2),keep_prob=0.8)

W3 = rand_init([64,10])
b3 = rand_init([10])
output = tf.matmul(layer2,W3)+b3

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=output)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output,1),
                                           tf.argmax(y_,1)),tf.float32))
trainstep = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
saver = tf.train.Saver()
with tf.Session() as sess:    
    if not os.path.exists("./savedata/model.ckpt.index"):
        sess.run(tf.global_variables_initializer())
        for i in range(20000):
            image,label = mnist.train.next_batch(50)
            trainstep.run({x_:image,y_:label})
            if i%1000==0:
                print("%d the accuracy is %.2f"%(i,accuracy.eval({x_:image,y_:label})))        
        savepath=saver.save(sess,"./savedata/model.ckpt")
        print("Model saved in the file: %s"%savepath)
    else:
        print("Trained model found")
        saver.restore(sess,"./savedata/model.ckpt")
    print("The test accuracy is %.2f"%accuracy.eval({x_:mnist.test.images,y_:mnist.test.labels}))



Trained model found
INFO:tensorflow:Restoring parameters from ./savedata/model.ckpt
The test accuracy is 0.94
